In [1]:
import sys
import os

from policies import SavedTFPolicy

path = os.path.dirname(os.path.abspath('__file__'))
sys.path.insert(0, path + '/../')
import tensorflow as tf
print(tf.__version__)
import tf_agents.policies
import tf_agents.specs
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
tf.config.set_visible_devices([], 'GPU')  #  allows testing during training
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from reinforcement_learning import labeling_functions
labeling_function = labeling_functions.get('Acrobot-v1')
from util.io.dataset_generator import map_rl_trajectory_to_vae_input
from util.io.dataset_generator import ErgodicMDPTransitionGenerator
from tf_agents.environments.wrappers import HistoryWrapper

2.4.1


In [2]:
py_env = suite_gym.load('Acrobot-v1')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(6,), dtype=tf.float32, name='observation', minimum=array([ -1.      ,  -1.      ,  -1.      ,  -1.      , -12.566371,
       -28.274334], dtype=float32), maximum=array([ 1.      ,  1.      ,  1.      ,  1.      , 12.566371, 28.274334],
      dtype=float32)))

In [18]:
n = 0
def display_labeling(trajectory):
    global n
    if n % 10 == 0:
        #input(trajectory.observation)
        print(trajectory.observation)
    n += 1

reward_metric = tf_metrics.AverageReturnMetric()
sac_policy_dir = '../saves/Acrobot-v1/dqn_policy'
saved_policy = SavedTFPolicy(sac_policy_dir)
dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env, saved_policy, num_episodes=5,
    observers=[
        display_labeling,
        lambda _: py_env.render(mode='human'),
        reward_metric
    ]).run()
reward_metric.result()

tf.Tensor([[ 0.99539465 -0.09586195  0.99519306  0.09793223  0.01382204  0.02670695]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.9337786 -0.3578513  0.7351449  0.67791   -0.5081879  1.5192906]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.99459106  0.10386809  0.6319394  -0.7750178  -1.7098955   2.2011163 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.45309705  0.8914612   0.3349677  -0.9422296   0.18711399 -1.4890157 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.9787037  -0.20527802  0.8125474   0.58289516  3.4630666  -5.0416017 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.05557732 -0.9984544  -0.47237778  0.8813962  -1.7536595   3.500368  ]], shape=(1, 6), dtype=float32)
tf.Tensor([[-0.02339064  0.9997264  -0.99909854  0.04245157 -1.671698   -1.1027225 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.99779093 -0.06643219  0.9504122   0.31099305 -0.30082437  0.80554897]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.98827046  0.15271369  0.65029514 -0.75968164 -0.9401164   1.350

<tf.Tensor: shape=(), dtype=float32, numpy=-64.8>

In [3]:
history_wrapper_env = HistoryWrapper(env=py_env)
tf_env = tf_py_environment.TFPyEnvironment(history_wrapper_env)
tf_env.time_step_spec()

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(3, 6), dtype=tf.float32, name='observation', minimum=array([ -1.      ,  -1.      ,  -1.      ,  -1.      , -12.566371,
       -28.274334], dtype=float32), maximum=array([ 1.      ,  1.      ,  1.      ,  1.      , 12.566371, 28.274334],
      dtype=float32)))

In [4]:
tf_env.current_time_step()

TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 3, 6), dtype=float32, numpy=
array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.99768126,  0.06805926,  0.99792546, -0.06438001,
          0.06003963, -0.04305384]]], dtype=float32)>)

In [6]:
labeling_function(tf_env.current_time_step().observation)

<tf.Tensor: shape=(1, 3, 7), dtype=bool, numpy=
array([[[False,  True,  True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True],
        [False,  True,  True,  True, False,  True, False]]])>

In [7]:
labeling_function(tf_env.current_time_step().observation)[:, -1, ...]

<tf.Tensor: shape=(1, 7), dtype=bool, numpy=array([[False,  True,  True,  True, False,  True, False]])>

In [8]:
tf_env.current_time_step().observation[:, -1, ...]

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[ 0.99768126,  0.06805926,  0.99792546, -0.06438001,  0.06003963,
        -0.04305384]], dtype=float32)>

In [11]:
labeling_function(tf_env.current_time_step().observation[:, -1, ...])

<tf.Tensor: shape=(1, 7), dtype=bool, numpy=array([[False,  True,  True,  True, False,  True, False]])>

In [12]:
x = tf_env.current_time_step().observation[:, -1, ...]

In [13]:
x

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[ 0.99768126,  0.06805926,  0.99792546, -0.06438001,  0.06003963,
        -0.04305384]], dtype=float32)>

In [14]:
x[:, -1, ...]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.04305384], dtype=float32)>